In [1]:
import keras
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, CuDNNLSTM, Bidirectional
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd /content/drive/'My Drive'/Colab

/content/drive/My Drive/Colab


In [0]:
#data
data = pd.read_csv('final_dataset.csv')
data  = data[19313:27006]

data.columns= list(range(32))
data=data.drop([0,1,12,13,14,15,16],axis=1)
data.columns=list(range(25))
input_feature= data.values
input_data = input_feature
sc= MinMaxScaler(feature_range=(0,1))
input_data[:,0:25] = sc.fit_transform(input_feature[:,:])

lookback= 168

test_size=int(.3 * len(data))
X=[]
y=[]
for i in range(len(data)-lookback-1):
    t=[]
    for j in range(0,lookback):
        
        t.append(input_data[[(i+j)], :])
    X.append(t)
    y.append(input_data[i+ lookback,3])
X, y= np.array(X), np.array(y)
X_test = X[:test_size+lookback]
X = X.reshape(X.shape[0],lookback, 25)
X_test = X_test.reshape(X_test.shape[0],lookback, 25)

In [5]:
#model architecture
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(units=175, return_sequences= True, input_shape=(X.shape[1],25))))
model.add(Dropout(.2))
model.add(Bidirectional(CuDNNLSTM(units=150, return_sequences=True)))
model.add(Dropout(.2))
model.add(Bidirectional(CuDNNLSTM(units=125, return_sequences = True)))
model.add(Dropout(.2))
model.add(Bidirectional(CuDNNLSTM(units=100, return_sequences= True)))
model.add(Dropout(.2))
model.add(Bidirectional(CuDNNLSTM(units=75, return_sequences=True)))
model.add(Dropout(.2))
model.add(Bidirectional(CuDNNLSTM(units=50, return_sequences = True)))
model.add(Dropout(.2))
model.add(Bidirectional(CuDNNLSTM(units=25)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

W0709 14:03:45.318689 140121124583296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0709 14:03:45.380294 140121124583296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:

#for j in range(1):
    #weights_name = str('bilstm7modelpyr')+str(f"{i:05d}")+'.h5'
    #model.load_weights(weights_name)
pred = model.predict(X)
pred_list = []
Y = []
for i in range(25):
    pred_list.append(pred)
    Y.append(y)

Y = np.array(Y).transpose()
Y = Y.reshape(7524,25)
Y = sc.inverse_transform(Y)
pred_array = np.array(pred_list).transpose()
pred_array = pred_array.reshape(7524,25)
unscaled_pred = sc.inverse_transform(pred_array)

r=sc.inverse_transform(input_data)
real = pd.DataFrame(r)
prediction = pd.DataFrame(unscaled_pred)
real_match = real[168:len(real)-1]
real_match = real_match.reset_index()
predvreal=prediction[3]-real_match[3]
real2pred=[]
real2real= []
err= []
for i in range(1,len(prediction)):
    real2pred.append(prediction[3][i]-real_match[3][i-1])
    real2real.append(real_match[3][i]-real_match[3][i-1])
    err.append((predvreal[i])**2)
print(sum(err))
"""
fig = plt.figure(figsize=(16,7))
ax1 = fig.add_subplot(1,1,1)
ax1.grid()
ax1.plot(predvreal.values, color= 'orange')
plt.title("Hourly Close price of Ethereum vs Prediction")
plt.xlabel("Time (latest-> oldest)")
plt.ylabel("ETH Close Price")
#plt.savefig('lstm_error.png')
fig.show()"""
#backtesting
predic=list(prediction[3]) # 3 is the column where close price for Ethereum is located
price=list(real_match[3])
wallet=[]
wallet.append(price[0])
for i in range(1,len(price)-1):
    if predic[i]>price[i-1]:
        wallet.append((price[i]-price[i-1])+wallet[i-1])
    else:
        wallet.append(wallet[i-1])
print('Start Amount: $',wallet[0],'Finishing Amount: $',wallet[-1])

6379.0158116866905
Start Amount: $ 287.76 Finishing Amount: $ 3327.600000000005


In [0]:
model.load_weights('bilstm7modelpyr2002.h5')

In [0]:
checkpoint = keras.callbacks.ModelCheckpoint('bilstm7modelpyr2_{epoch:03d}.h5',monitor='loss', mode='min',save_best_only=True,save_weights_only= True, period=1) 
history = model.fit(X, y, epochs=2200, batch_size=200, callbacks=[checkpoint])
plt.figure(figsize=(30,10))
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

Epoch 1/2200
7524/7524 [==============================] - 29s 4ms/step - loss: 1.4078e-05
Epoch 2/2200
7000/7524 [==========================>...] - ETA: 2s - loss: 1.3220e-05

In [0]:
keras.callbacks.History()